# Tutorial: Ferromagnetic Resonance (FMR) Spectrum Analysis with Python

In this tutorial, we will guide you through the process of analyzing Ferromagnetic Resonance (FMR) spectra using Python. We will cover loading experimental data, filtering, Full Width Half Maximum (FWHM) analysis, Lorentzian fitting, and linear regression for line width estimation.

## Prerequisites

- Python installed on your machine.
- Required Python libraries: `matplotlib`, `numpy`, `scipy`, `pandas`.

You can install these libraries using:

```bash
pip install matplotlib numpy scipy pandas
```

## Step 1: Setup and Data Loading

```python
import matplotlib.pyplot as plt
import os
from func import filters as fil
from func import Crop, Calc, Load, Fitting, FWHM

plt.style.use('seaborn-v0_8')
import warnings
warnings.filterwarnings('ignore')

# Load 1-Port data
dir = r'c:\Users\plyslab\Desktop\VNA Automation\Data\Spin_pumping\1_Port_MgO5NiFe20Mgo15_0o_0.3-15.0_GHz_@_-1000.0-1000.0_Oe'
Info, freq, field, RefS11, S11 = Load.NPZ_1P(dir)
```

## Step 2: Set Frequency and Field Bounds

```python
# Set frequency and field bounds
lower_freq = 1.5
upper_freq = 5
lower_field = -400
upper_field = 400

freq_used = Crop.cut_freq(freq, lower_freq, upper_freq)[0]
lower_freq = Crop.cut_freq(freq, lower_freq, upper_freq)[1]
upper_freq = Crop.cut_freq(freq, lower_freq, upper_freq)[2]

field_used = Crop.cut_field(field, lower_field, upper_field)[0]
lower_field = Crop.cut_freq(field, lower_field, upper_field)[1]
upper_field = Crop.cut_freq(field, lower_field, upper_field)[2]
```

## Step 3: Filtering Data

```python
# Calculate the absorbed power
abspow = Calc.AbsPower1P(S11, RefS11)

# Apply filtering functions
filteredData = fil.Filter_Spline(abspow, a=0.1)
filteredData = fil.FilterBG_Median(filteredData)
filteredData = fil.FilterBG_MedianH(filteredData)

# Crop the data
cutData = filteredData[lower_freq:upper_freq, lower_field:upper_field]
```

## Step 4: Full Width Half Maximum (FWHM) Analysis
- The function for FWHM anaylsis has been stored in the `FWHM` class in the `func` module.

```python
# Perform FWHM analysis for specific frequencies
FWHM.fwhm(cutData, field_used, frequency=2, freq_used=freq_used)
FWHM.fwhm(cutData, field_used, frequency=4, freq_used=freq_used)
```

## Step 5: Lorentzian Fitting
- We use the Lorentzian equation defined in the `Fitting` class of the `func` module

```python
# Specify output file path for fitting results
sample = 'NiFe_test'
file_name = f'1Port {sample} Fitting_{freq_used.min()}-{freq_used.max()} GHz'
filepath = os.path.join(r'C:\Users\plyslab\Desktop\VNA Automation\Fittings', f'{file_name}.csv')

# Perform Lorentzian fitting and save results
Fitting.lorentzian(freq_used, field_used, cutData, filepath, plotting=True)
```

## Step 6: Spectrum Visualization
- Visualize the FMR spectrum of both peaks using the Lorentzian fitted data.

```python
# Visualize Lorentzian fitting results
import pandas as pd
import numpy as np

# Read the CSV file
fit = pd.read_csv(filepath)

# Extract columns
Hfmr1 = fit.loc[:, 'H_fmr 1'].values
Hfmr2 = fit.loc[:, 'H_fmr 2'].values
deltaH1 = fit.loc[:, 'DeltaH 1']
deltaH2 = fit.loc[:, 'DeltaH 2']
f = fit.loc[:, 'Frequency (GHz)']

# Filter out outliers
f1_filtered = f[~outliers(Hfmr1)]
f2_filtered = f[~outliers(Hfmr2)]
Hfmr1_filtered = Hfmr1[~outliers(Hfmr1)]
Hfmr2_filtered = Hfmr2[~outliers(Hfmr2)]

# Plot the data
plt.figure(figsize=(8, 6))
plt.plot(Hfmr1_filtered, f1_filtered, 'ko', markersize=4, label='1st Peak')
plt.plot(Hfmr2_filtered, f2_filtered, 'ro', markersize=4, label='2nd Peak')
plt.xlabel('Frequency (GHz)')
plt.ylabel('$H_{fmr}$')
plt.legend()
plt.grid(True)
```

## Step 7: Linear Regression

```python
# Perform linear regression for the 1st peak
slope, intercept, _, _, _ = linregress(f, deltaH1)
fit_line = slope * f + intercept

# Filter outliers
outliers = abs(deltaH1 - fit_line) > 3 * np.std(deltaH2)

# Plot the linear fit without outliers
plt.figure(figsize=(8, 6))
plt.plot(f, deltaH1, 'ko', markersize=5, label='Experimental')
plt.plot(f, fit_line_filtered, 'r-', label='Fit Line without Outliers')
plt.plot(f[outliers], deltaH1[outliers], 'bo', label='Outliers', markerfacecolor='none', markeredgewidth=1)
plt.title('Linear Fitting (1st Peak)')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Line Width (Oe)')
plt.legend()
plt.grid(False)
plt.show()

# Linear Regression Results
print('Results:\n')
print(f'Slope: {slope_filtered:.4f}')
print(f'Intercept: {intercept_filtered:.4f}')

gamma = 17.6E-3
alpha = (gamma * slope_filtered) / (2 * np.pi)
print(f'alpha: {alpha: .4e}')
```

Repeat the linear regression steps for the 2nd peak. Adjust parameters as needed for your specific data and analysis.

This tutorial covers the essential steps for FMR spectrum analysis using Python. You can adapt and extend this code for your specific experimental data and requirements.

_________________________________________

**Developer:** Mahad Naveed

**Supervisor:** Dr. Sabieh Anwar

**Mentor:** Dr. Adnan Raza